In [1]:
import torch 
import torch.nn.functional as F 
from transformers import AutoModelForCausalLM, AutoTokenizer
# On choisit un modèle très léger (environ 500 Mo)
model_name = "facebook/opt-125m" 

print("Téléchargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Téléchargement du modèle...")
model = AutoModelForCausalLM.from_pretrained(model_name)

# On déplace le modèle sur le GPU si disponible, sinon CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


print(f"Modèle chargé sur {device} !")


c:\Users\BONETTI\Desktop\3A\Intro to generative models\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Téléchargement du tokenizer...
Téléchargement du modèle...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Modèle chargé sur cpu !


In [2]:
import random 
import numpy as np 
import torch

def compute_log_likelihood(model, tokenizer, sequence):
    #Calcule la log-vraisemblance log(p(x)) d'une séquence
    inputs = tokenizer(sequence, return_tensors="pt").to(model.device)
    input_ids = inputs["input_ids"]
    
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        # Le modèle renvoie déjà la CrossEntropy (Negative Log Likelihood moyenne)
        # On la multiplie par le nombre de tokens pour avoir la somme des log-probs
        log_p_x = -outputs.loss.item() * (input_ids.shape[1] - 1)
        
    return log_p_x, inputs

def run_correction_tracker(model, tokenizer, initial_text, alpha=16.0, steps=20, block_size=15):
    current_text = initial_text
    # Calcul initial
    current_log_p, _ = compute_log_likelihood(model, tokenizer, current_text)
    
    history = []
    history.append({"step": 0, "text": current_text, "log_p": current_log_p, "status": "Initial"})

    print(f"Départ: {current_text} | Log P: {current_log_p:.2f}")

    for i in range(1, steps + 1):
        # 1. On transforme le texte en IDs pour manipuler les blocs
        input_ids = tokenizer.encode(current_text, return_tensors="pt").to(model.device)
        seq_len = input_ids.shape[1]

        # 2. Choisir un bloc à modifier (on évite le tout début)
        start_idx = random.randint(min(5, seq_len-1), max(5, seq_len - block_size - 1))
        prefix_ids = input_ids[:, :start_idx]

        # 3. Proposer un nouveau bloc (x')
        with torch.no_grad():
            new_block_ids = model.generate(
                prefix_ids, 
                max_new_tokens=block_size, 
                do_sample=True, 
                temperature=1.0, # On échantillonne normalement
                attention_mask=prefix_ids.ne(tokenizer.pad_token_id).long(),
                pad_token_id=tokenizer.eos_token_id
            )
        
        proposed_text = tokenizer.decode(new_block_ids[0], skip_special_tokens=True)
        #print(f"Phrase complète à cette étape : \n {proposed_text}")
        proposed_log_p, _ = compute_log_likelihood(model, tokenizer, proposed_text)

        # 4. Calcul du ratio Metropolis-Hastings
        # log(A) = alpha * (log_p_proposed - log_p_current)
        acceptance_log_ratio = alpha * (proposed_log_p - current_log_p)
        
        accepted = False
        if np.log(random.random()) < acceptance_log_ratio:
            current_text = proposed_text
            current_log_p = proposed_log_p
            accepted = True

        status = "ACCEPTÉ" if accepted else "REJETÉ"
        print(f"Étape {i}: {status} | Nouveau texte: {proposed_text[:50]}... | Log P: {proposed_log_p:.2f}")
        
        history.append({
            "step": i, 
            "text": proposed_text, 
            "log_p": proposed_log_p, 
            "status": status,
            "final_text_at_step": current_text
        })

    return history



In [3]:
# --- TEST ---
# On commence volontairement avec une phrase un peu bancale
prompt_initial = "A prime number is a number that has three divisors: 1, itself and"
tracker_results = run_correction_tracker(model, tokenizer, prompt_initial, alpha=16, steps=20)

Départ: A prime number is a number that has three divisors: 1, itself and | Log P: -66.02
Étape 1: ACCEPTÉ | Nouveau texte: A prime number is typically a prime number. For ex... | Log P: -54.62
Étape 2: REJETÉ | Nouveau texte: A prime number is a good number for when to pick u... | Log P: -64.50
Étape 3: REJETÉ | Nouveau texte: A prime number is a very short answer to a simple ... | Log P: -56.24
Étape 4: ACCEPTÉ | Nouveau texte: A prime number is 2 divided by 1.
2
Calculate the ... | Log P: -35.80
Étape 5: REJETÉ | Nouveau texte: A prime number is the key to a great man. The worl... | Log P: -67.01
Étape 6: REJETÉ | Nouveau texte: A prime number is a number used to denote a number... | Log P: -55.44
Étape 7: REJETÉ | Nouveau texte: A prime number is 1.999999.
There is no prime numb... | Log P: -60.43


c:\Users\BONETTI\Desktop\3A\Intro to generative models\.conda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\BONETTI\.cache\huggingface\hub\models--facebook--opt-125m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Étape 8: REJETÉ | Nouveau texte: A prime number is a number given to an arrangement... | Log P: -60.13
Étape 9: REJETÉ | Nouveau texte: A prime number is 3.
It's the number I am willing ... | Log P: -64.04
Étape 10: REJETÉ | Nouveau texte: A prime number is 5*(1 - 6/(-19)). Suppose 2*x =... | Log P: -53.00
Étape 11: REJETÉ | Nouveau texte: A prime number is the number of people you see a w... | Log P: -53.66
Étape 12: REJETÉ | Nouveau texte: A prime number is used with confidence when descri... | Log P: -74.34
Étape 13: REJETÉ | Nouveau texte: A prime number is a real number, and therefore it'... | Log P: -62.02
Étape 14: REJETÉ | Nouveau texte: A prime number is just a few hundred digits.
>A pr... | Log P: -43.47
Étape 15: REJETÉ | Nouveau texte: A prime number is one set of numbers of digits, li... | Log P: -61.40
Étape 16: REJETÉ | Nouveau texte: A prime number is a small number of the number one... | Log P: -65.80
Étape 17: REJETÉ | Nouveau texte: A prime number is a number, not a 